在机器学习中，数值特征预处理是模型训练前的关键步骤，其核心目标是通过调整数据的尺、分布或补全缺失值，让数据更符合模型的假设（如线性模型假设特征分布相近），从而提升模型的稳定性和性能。
- 归一化
- 标准化
- 缺失值处理

# 归一化

- 归一化是将特征值缩放到一个固定范围（如[0, 1]），常用方法包括Min-Max缩放和Z-Score标准化。
- 归一化的目标是消除特征之间的量纲差异，使模型对特征的影响更加均衡。

In [12]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# 创建示例数据
data = np.array([1, 2, 3, 4, 5], dtype=float)

# 简单归一化：缩放到 [0, 1]
# normalized = (data - data.min()) / (data.max() - data.min())

# 使用MinMaxScaler进行归一化
scaler = MinMaxScaler(feature_range=(0, 1))# 归一化范围为[0, 1]; 
                        #feature_range=(-1, 1)# 归一化范围为[-1, 1]
normalized = scaler.fit_transform(data.reshape(-1, 1)).flatten()# 归一化后的数据类型为float64

print("原始数据:", data)
print("归一化范围:", normalized.min(), "到", normalized.max())
print("归一化后:", normalized)
print("归一化后的数据类型:", normalized.dtype)
print("归一化后的数据形状:", normalized.shape)
print("归一化后的数据元素数量:", normalized.size)

原始数据: [1. 2. 3. 4. 5.]
归一化范围: 0.0 到 1.0
归一化后: [0.   0.25 0.5  0.75 1.  ]
归一化后的数据类型: float64
归一化后的数据形状: (5,)
归一化后的数据元素数量: 5


归一化的优点:
- 归一化后的特征在相同的尺度上，避免了特征之间的量纲问题。
- 归一化后的特征在相同的区间内，方便模型的训练和解释。
-  归一化处理之后，更容易通过梯度下降法找最优解

归一化的缺点:
- 归一化容易受到异常点的影响
- 归一化处理数据，鲁棒性较差（也就是在异常点的影响下波动较大），只适合精确、数据量小的场景
- 归一化会改变原始数据的分布，某些异常值可能会被压缩到正常范围内。
- 归一化会丢失原始数据的一些信息，如数据的中心位置和数据的分布形状。


# 标准化

标准化的核心是将特征转换为均值为0、标准差为1的分布（也就是分布），保留数据的相对离散程度，更适合近似正态分布的数据。
标准化也被称力（Z-score缩放）

不会破坏数据的自然分布

In [17]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# 创建示例数据
data = np.array([1, 2, 3, 4, 5], dtype=float)

# 简单标准化
# 标准化数据：减去均值后除以标准差
# mean = np.mean(data)
# std = np.std(data)
# normalized_data = (data - mean) / std
# print("原始数据:", data)
# print("标准化后:", normalized_data)

# 使用StandardScaler
model = StandardScaler()
normalized_data = model.fit_transform(data.reshape(-1, 1))

print("原始数据:", data)
print("标准化后:", normalized_data)
print("标准化参数:", model.mean_, model.scale_)
print("方差:", model.var_)
print("标准差:", model.scale_)


原始数据: [1. 2. 3. 4. 5.]
标准化后: [[-1.41421356]
 [-0.70710678]
 [ 0.        ]
 [ 0.70710678]
 [ 1.41421356]]
标准化参数: [3.] [1.41421356]
方差: [2.]
标准差: [1.41421356]


# 缺失值处理

In [ ]:
import pandas as pd
import numpy as np

# 1. 删除缺失值
def drop_missing(df: pd.DataFrame, axis: int = 0, how: str = 'any') -> pd.DataFrame:
    """
    删除缺失值
    :param df: 原始数据框
    :param axis: 0 删除行，1 删除列
    :param how: 'any' 只要存在缺失就删除，'all' 全部缺失才删除
    :return: 处理后的数据框
    """
    return df.dropna(axis=axis, how=how)


# 2. 均值/中位数/众数填充
def fill_with_statistic(df: pd.DataFrame, strategy: str = 'mean') -> pd.DataFrame:
    """
    用统计量填充数值型缺失值
    :param df: 原始数据框
    :param strategy: 'mean' 均值，'median' 中位数，'mode' 众数
    :return: 处理后的数据框
    """
    df_copy = df.copy()
    for col in df_copy.select_dtypes(include=np.number).columns:
        if strategy == 'mean':
            df_copy[col].fillna(df_copy[col].mean(), inplace=True)
        elif strategy == 'median':
            df_copy[col].fillna(df_copy[col].median(), inplace=True)
        elif strategy == 'mode':
            df_copy[col].fillna(df_copy[col].mode()[0], inplace=True)
    return df_copy


# 3. 前向/后向填充
def fill_with_ffill_bfill(df: pd.DataFrame, method: str = 'ffill') -> pd.DataFrame:
    """
    用前向或后向填充缺失值
    :param df: 原始数据框
    :param method: 'ffill' 前向填充，'bfill' 后向填充
    :return: 处理后的数据框
    """
    return df.fillna(method=method)


# 4. 常数填充
def fill_with_constant(df: pd.DataFrame, fill_value: any) -> pd.DataFrame:
    """
    用指定常数填充缺失值
    :param df: 原始数据框
    :param fill_value: 用于填充的常数
    :return: 处理后的数据框
    """
    return df.fillna(fill_value)


# 5. 插值法填充
def fill_with_interpolate(df: pd.DataFrame, method: str = 'linear') -> pd.DataFrame:
    """
    用插值法填充缺失值
    :param df: 原始数据框
    :param method: 插值方法，如 'linear', 'polynomial', 'spline' 等
    :return: 处理后的数据框
    """
    return df.interpolate(method=method)


# 6. KNN填充
from sklearn.impute import KNNImputer

def fill_with_knn(df: pd.DataFrame, n_neighbors: int = 5) -> pd.DataFrame:
    """
    用KNN填充缺失值
    :param df: 原始数据框
    :param n_neighbors: 邻居数量
    :return: 处理后的数据框
    """
    imputer = KNNImputer(n_neighbors=n_neighbors)
    return pd.DataFrame(imputer.fit_transform(df), columns=df.columns)


# 7. 多重插补（MICE）
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

def fill_with_mice(df: pd.DataFrame, max_iter: int = 10) -> pd.DataFrame:
    """
    用多重插补（MICE）填充缺失值
    :param df: 原始数据框
    :param max_iter: 最大迭代次数
    :return: 处理后的数据框
    """
    imputer = IterativeImputer(max_iter=max_iter, random_state=42)
    return pd.DataFrame(imputer.fit_transform(df), columns=df.columns)


# 8. 分类变量用特殊值填充
def fill_categorical_with_missing(df: pd.DataFrame, fill_value: str = 'Missing') -> pd.DataFrame:
    """
    对分类变量缺失值用特殊值填充
    :param df: 原始数据框
    :param fill_value: 用于填充的特殊值
    :return: 处理后的数据框
    """
    df_copy = df.copy()
    for col in df_copy.select_dtypes(include='object').columns:
        df_copy[col].fillna(fill_value, inplace=True)
    return df_copy


# 9. 分组填充（按类别均值填充）
def fill_groupby_mean(df: pd.DataFrame, group_cols: list, target_col: str) -> pd.DataFrame:
    """
    按分组均值填充缺失值
    :param df: 原始数据框
    :param group_cols: 分组列名列表
    :param target_col: 需要填充的目标列
    :return: 处理后的数据框
    """
    df_copy = df.copy()
    df_copy[target_col] = df_copy.groupby(group_cols)[target_col].transform(
        lambda x: x.fillna(x.mean())
    )
    return df_copy


# 10. 模型预测填充（示例：用随机森林）
from sklearn.ensemble import RandomForestRegressor

def fill_with_model(df: pd.DataFrame, target_col: str) -> pd.DataFrame:
    """
    用模型预测缺失值（以随机森林为例）
    :param df: 原始数据框
    :param target_col: 需要填充的目标列
    :return: 处理后的数据框
    """
    df_copy = df.copy()
    missing_mask = df_copy[target_col].isnull()
    if missing_mask.sum() == 0:
        return df_copy

    # 分离训练集和预测集
    train = df_copy[~missing_mask]
    test = df_copy[missing_mask]

    # 特征列（排除目标列）
    feature_cols = [c for c in df_copy.columns if c != target_col]

    # 训练模型
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(train[feature_cols], train[target_col])

    # 预测缺失值
    predicted = model.predict(test[feature_cols])
    df_copy.loc[missing_mask, target_col] = predicted
    return df_copy

